In [1]:
import numpy as np

In [2]:
class OptimalControlProblem(object):
    def __init__(self, t0 = 0.0, tf=50.0, dym_dim = 3, con_dim = 2,
               S0 = 174673.0, I0 = 132.0, R0 = 0.0):

        #Parameters
        self.t0 = t0
        self.tf = tf
        self.beta_d = 0.75
        self.Delta = 0.03
        self.beta_e = 1.5e-10
        self.alpha_ = 0.5
        self.mu_c = 1e-6
        self.b = 1.2e-5
        self.mu_v = 5e-5
        self.gamma = 3e-3
        self.rho = 0.8

        #positive weight constant
        self.b1 = 2.0
        self.w1 = 1e8
        self.w2 = 1e9
        
        #Initial conditions   
        self.S0 = S0
        self.I0 = I0
        self.R0 = R0
        self.lambda_final = np.zeros([1,3])

    def set_parameters(self, beta_d, Delta, beta_e, alpha_, mu_c, b, mu_v, 
                       gamma, rho, b1, w1, w2, S0, I0, R0):
        
        self.beta_d = beta_d
        self.Delta = Delta
        self.beta_e = beta_e
        self.alpha_ = alpha_
        self.mu_c = mu_c
        self.mu_v = mu_v
        self.gamma = gamma
        self.rho = rho

        self.b1 = b1
        self.w1 = w1
        self.w2 = w2

        self.S0 = S0
        self.I0 = I0
        self.R0 = R0

    def g(self, x_k, u_k):

        beta_d = self.beta_d
        Delta = self.Delta
        beta_e = self.beta_e
        alpha_ = self.alpha_
        mu_c = self.mu_c
        mu_v = self.mu_v
        gamma = self.gamma
        rho = self.rho

        S = x_k[0,0]
        I = x_k[0,1]
        R = x_k[0,2]

        u1 = u_k[0,0]
        u2 = u_k[0,1]

        term1 = (beta_d*S*I)/(S+I)
        term2 = (alpha_*I)/(1+b*I)
        dSdt = Delta - term1 - beta_e*S*I - mu_c*S + term2 - (u1 + rho*u2)*S
        dIdt = term1 + beta_e*S*I - term2 - (mu_c+mu_v+gamma)*I - u1*I
        dRdt = u1*(S+I) + rho*u2*S + gamma*I - mu_c*R

        ode = np.array([dSdt, dIdt, dRdt])
        ode = ode.reshape([1,3])
        return ode

    def lambda_function(self, x_k, u_k, lambda_k):
        b1 = self.b1

        beta_d = self.beta_d
        Delta = self.Delta
        beta_e = self.beta_e
        alpha_ = self.alpha_
        mu_c = self.mu_c
        mu_v = self.mu_v
        gamma = self.gamma
        rho = self.rho

        S = x_k[0,0]
        I = x_k[0,1]
        R = x_k[0,2]

        u1 = u_k[0,0]
        u2 = u_k[0,1]

        lambda1 = lambda_k[0, 0]
        lambda2 = lambda_k[0, 1]
        lambda3 = lambda_k[0, 2]

        l_term1 = (S+I) ** 2
        l_term2 = (1+b*I) ** 2

        lambdaprime1 = (lambda1-lambda2)*I*((beta_d*I)/l_term1+beta_e) + lambda1*mu_c \
                        + (u1 + rho*u2)*(lambda1 - lambda3)
        lambdaprime2 = -b1 + (lambda1-lambda2)*S*(((beta_d*S)/l_term1+beta_e) - alpha_/l_term2) \
                        + lambda2*(mu_c + mu_v + gamma) + (lambda2 - lambda3)*u1 - lambda3*gamma
        lambdaprime3 = lambda3*mu_c

        lambdaprime = np.array([lambdaprime1, lambdaprime2, lambdaprime3])
        lambdaprime = lambdaprime.reshape([1,3])
        return lambdaprime

    def optimality_condition(self, x_k, u_k, lambda_k, n_max):

        w1 = self.w1
        w2 = self.w2

        rho = self.rho

        S = x_k[:,0]
        I = x_k[:,1]

        lambda1 = lambda_k[:, 0]
        lambda2 = lambda_k[:, 1]
        lambda3 = lambda_k[:, 2]

        aux1 = ((lambda1-lambda3)*S + (lambda2-lambda3)*I)/w1
        aux2 = (rho*(lambda1-lambda3)*S)/w2

        pp1 = np.max([np.zeros(n_max), aux1], axis=0)
        pp2 = np.max([np.zeros(n_max), aux2], axis=0)

        u_aster1 = np.min([np.ones(n_max), pp1], axis=0)
        u_aster2 = np.min([np.ones(n_max), pp2], axis=0)

        u_aster = np.zeros([n_max, 2])
        u_aster[:,0] = u_aster1
        u_aster[:,1] = u_aster2

        return u_aster

In [3]:
class ForwardBackwardSweep(OptimalControlProblem):
    def __init__(self, eps=.0001, n_max=1000):

        super(ForwardBackwardSweep, self).__init__()
        self.n_max = n_max
        self.eps = eps
        self.t = np.linspace(self.t0, self.tf, n_max)
        self.h = self.t[1] - self.t[0]
        self.x = np.zeros([n_max, 3])
        self.u = np.zeros([n_max, 2])
        self.lambda_adjoint = np.zeros([n_max, 3])

    def runge_kutta_forward(self, u):
        x0 = np.array([self.S0, self.I0, self.R0])

        h = self.h
        n_max = self.n_max
        sol = np.zeros([n_max, 3])
        sol[0] = x0

        for j in np.arange(n_max - 1):
            x_j = sol[j].reshape([1,3])
            u_j = u[j].reshape([1,2])
            u_jp1 = u[j+1].reshape([1,2])
            u_mj = 0.5*(u_j + u_jp1)

            k1 = self.g(x_j, u_j)
            k2 = self.g(x_j*0.5*h*k1, u_mj)
            k3 = self.g(x_j*0.5*h*k2, u_mj)
            k4 = self.g(x_j + h*k3, u_jp1)

            sol[j+1] = x_j + (h/6.0) + (k1 + 2*k2 + 2*k3 + k4)
            self.x = sol
        return sol
        
    def runge_kutta_backward(self, x, u):
        lambda_final = self.lambda_final
        h = self.h
        n_max = self.n_max
        sol = np.zeros([n_max, 3])
        sol[-1] = lambda_final

        for j in np.arange(n_max-1, 0, -1):
            lambda_j = sol[j].reshape([1,3])
            x_j = x[j].reshape([1,3])
            x_jm1 = x[j-1].reshape([1,3])
            x_mj = 0.5*(x_j + x_jm1)
            u_j = u[j].reshape([1,2])
            u_jm1 = u[j-1].reshape([1,2])
            u_mj = 0.5*(u_j + u_jm1)

            k1 = self.lambda_function(x_j, u_j, lambda_j)
            k2 = self.lambda_function(x_mj, u_mj, lambda_j-0.5*h*k1)
            k3 = self.lambda_function(x_mj, u_mj, lambda_j-0.5*h*k2)
            k4 = self.lambda_function(x_jm1, u_jm1, lambda_j-h*k3)

            sol[j - 1] = lambda_j - (h / 6.0) * (k1 + 2 * k2 + 2 * k3 + k4)
            self.lambda_adjoint = sol
        return sol

    def forward_backward_sweep(self):
        flag = True
        cont = 1
        eps = self.eps
        x = self.x
        u = self.u
        n_max = self.n_max
        lambda_ = self.lambda_adjoint

        while flag:
            u_old = u
            x_old = x
            lambda_old = lambda_
            
            x = self.runge_kutta_forward(u)
            lambda_ = self.runge_kutta_backward(x,u)
            u_1 = self.optimality_condition(x,u, lambda_, n_max)
            alpha = 0.5
            u = alpha*u_1 + (1.0-alpha)*u_old

            test_1 = np.linalg.norm(u_old-u, 1)*(np.linalg.norm(u,1)**(-1))
            test_2 = np.linalg.norm(x_old-x, 1)*(np.linalg.norm(x,1)**(-1))
            test_3 = np.linalg.norm(lambda_old-lambda_, 1)*(np.linalg.norm(lambda_,1)**(-1))

            test = np.max([test_1, test_2, test_3])
            flag = (test > eps)
            cont = cont + 1
            print(cont, test)
            return [x, lambda_, u]

In [4]:
b1 = 2.0
w1 = 1e8
w2 = 1e9


beta_d = 0.75
Delta = 0.03
beta_e = 1.5e-10
alpha_ = 0.5
mu_c = 1e-6
b = 1.2e-5
mu_v = 5e-5
gamma = 3e-3
rho = 0.8

S0 = 174673.0
I0 = 132.0
R0 = 0.0

In [6]:
fbsm = ForwardBackwardSweep()

fbsm.set_parameters(beta_d, Delta, beta_e, alpha_, mu_c, b, mu_v, 
                   gamma, rho, b1, w1, w2, S0, I0, R0)

t = fbsm.t

x_wc = fbsm.runge_kutta_forward(fbsm.u)
[x, lambda_, u] = fbsm.forward_backward_sweep()

2 nan


C:\Users\Admin\AppData\Local\Temp\ipykernel_18652\2293086213.py:67: RuntimeWarning: divide by zero encountered in scalar divide
  term1 = (beta_d*S*I)/(S+I)
C:\Users\Admin\AppData\Local\Temp\ipykernel_18652\2293086213.py:67: RuntimeWarning: invalid value encountered in scalar add
  term1 = (beta_d*S*I)/(S+I)
C:\Users\Admin\AppData\Local\Temp\ipykernel_18652\2293086213.py:68: RuntimeWarning: invalid value encountered in scalar divide
  term2 = (alpha_*I)/(1+b*I)
C:\Users\Admin\AppData\Local\Temp\ipykernel_18652\2293086213.py:69: RuntimeWarning: invalid value encountered in scalar multiply
  dSdt = Delta - term1 - beta_e*S*I - mu_c*S + term2 - (u1 + rho*u2)*S
C:\Users\Admin\AppData\Local\Temp\ipykernel_18652\2293086213.py:70: RuntimeWarning: invalid value encountered in scalar multiply
  dIdt = term1 + beta_e*S*I - term2 - (mu_c+mu_v+gamma)*I - u1*I
C:\Users\Admin\AppData\Local\Temp\ipykernel_18652\2293086213.py:71: RuntimeWarning: invalid value encountered in scalar add
  dRdt = u1*(S+I